<H1>Neighborhoods in Toronto</H1>

<p>Imports...</p>


In [50]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

<p>Loading data</p>

In [16]:
doc_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(doc_url).text
soup = BeautifulSoup(source, 'lxml')

<p>Looking for the table and diving into its body</p>

In [185]:
#Since there is only one table and its class is "wikitable sortable", let's extract it
table = soup.find("table",class_="wikitable sortable")
table_body = soup.find("tbody")


<h2>Extraction of data in a dictionnary</h2>
<p>The main assumption is to remove ant line where borough OR neighborough contained "Not assigned". It is inline with the first instruction but seems to contradict the instruction about Queen's Park where neighborough ony had "Not assigned. <b>This makes one less row in the Dataframe at the end.</b></p>

In [194]:
#Extract all raws
table_rows = table_body.find_all("tr")
# Preparing a dictionnary with postal code as key
cp_dict=dict()

# Loop over postocde list excluding the fist line (title)
for wiki_postcode in table_rows[1:]:
    postcode_data = wiki_postcode.find_all("td")
    # Removong rows with not assigned entries
    if postcode_data[1].text.rstrip() == "Not assigned" or postcode_data[2].text.rstrip()== "Not assigned" :    
        continue
  
    postcode = postcode_data[0].text
    # Process the case where there is a href or not in borough description
    if postcode_data[1].find("a"):
        borough = postcode_data[1].find("a").text
    else:
        borough = postcode_data[1].text

    neighborough_data = postcode_data[2].text.rstrip()

    # Update dictionnary according if there is already an entry for the postal code or if it needs to be created
    if postcode in cp_dict:
        cp_dict[postcode][1]=cp_dict[postcode][1]+","+neighborough_data
    else:
        cp_dict[postcode]=[borough,neighborough_data]

<h2>Create DataFrame from dictionnary</h2>

In [195]:
#Change dictionary into a list
cplist=[]
for i in cp_dict.keys():
  cplist.append((i,cp_dict[i][0],cp_dict[i][1]))
# Load Dataframe from list
cols = ["Postcode", "Borough", "Neiborough"]
cp = pd.DataFrame.from_records(cplist, columns=cols)


<h2>Display the shape of the DataFrame and save it</h2>

In [196]:
print(cp.shape)
cp.to_csv("Toronto.csv")
cp.head()

(102, 3)


,Postcode,Borough,Neiborough
0,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf..."
1,M9M,North York,"Emery,Humberlea"
2,M1R,Scarborough,"Maryvale,Wexford"
3,M1T,Scarborough,"Clarks Corners,Sullivan,Tam O'Shanter"
4,M3C,North York,"Flemingdon Park,Don Mills South"
